In [23]:
%load_ext autoreload
%autoreload 2

In [45]:
import sys
import os

script_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

import src.config as config
from dotenv import load_dotenv
from src.paths import PARENT_DIR

In [48]:
load_dotenv(PARENT_DIR / '.env')
os.environ['HOPSWORKS_API_KEY']

'5RAv3V2iX0OsO1GO.3x42EyzaRkF7HxPPgKHS2TWoKrFmPFkFnjy7FRD3eEK3qo8geC7lS4zMPcHr6lEJ'

In [2]:

from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2024-03-30 03:00:00')


In [3]:
fetch_data_from - timedelta(days=7*52)

Timestamp('2023-03-04 03:00:00')

In [4]:
fetch_data_to, fetch_data_from

(Timestamp('2024-03-30 03:00:00'), Timestamp('2024-03-02 03:00:00'))

In [5]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [6]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2024-03-02 03:00:00'), to_date_=Timestamp('2023-04-01 03:00:00')
File 2023-03 was already in local storage
File 2023-04 was already in local storage


In [7]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:00<00:00, 515.48it/s]


In [8]:
ts_data.head()

,pickup_hour,rides,pickup_location_id
0,2024-03-02 03:00:00,0,1
1,2024-03-02 04:00:00,0,1
2,2024-03-02 05:00:00,2,1
3,2024-03-02 06:00:00,1,1
4,2024-03-02 07:00:00,0,1


In [ ]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [28]:
os.environ['HOPSWORKS_API_KEY']

'5RAv3V2iX0OsO1GO.3x42EyzaRkF7HxPPgKHS2TWoKrFmPFkFnjy7FRD3eEK3qo8geC7lS4zMPcHr6lEJ'

In [9]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group >> where i want to store the data
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/variables/docker_base_image_python_version). Server response: 
HTTP code: 401, HTTP reason: Unauthorized, body: b'{"errorCode":320013,"errorMsg":"API key not found in the database"}', error code: 320013, error msg: API key not found in the database, user msg: 

In [ ]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})